In [1]:
import pandas as pd
import datetime

In [2]:
fakespace_df = pd.read_csv("01-tw-fakespaces.csv")

# Incorporating rdf transformation

In [3]:
#define key namespaces to be used in the mapping
famo = "http://ontology.eil.utoronto.ca/FAMO/famo/"
ex = "http://example.com/data/"
rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
owl = "http://www.w3.org/2002/07/owl#"

#current datetime to be used for identifier suffix
dt_now = datetime.datetime.now().strftime("%d%b%Y_%H%M%S")

## Role mapping

**TODO** test for null values in all columns before mapping

In [4]:
ttl_list = []
#prefixes
ttl_list.append(f"PREFIX famo: <{famo}>"+'\n')
ttl_list.append(f"PREFIX ex: <{ex}>"+'\n')
ttl_list.append(f"PREFIX rdf: <{rdf}>"+'\n')
ttl_list.append(f"PREFIX owl: <{owl}>"+'\n')

for i in range(0,len(fakespace_df)):
    #record instantiation and interpretation
    ttl_list.append(f"ex:wms_spacerec_{fakespace_df['Space'][i]} rdf:type famo:Record."+'\n')
    #instantiated in wms
    ttl_list.append(f"ex:wms_spacerec_{fakespace_df['Space'][i]} famo:instantiatedIn ex:wms_datasystem."+'\n')
    ttl_list.append(f"ex:wms_spacerec_{fakespace_df['Space'][i]} famo:hasInterpretation ex:wms_spaceinterpretation_{fakespace_df['Space'][i]}."+'\n')
    ttl_list.append(f"ex:wms_spaceinterpretation_{fakespace_df['Space'][i]} famo:denotes ex:{fakespace_df['Space'][i]}."+'\n')
    ttl_list.append(f"ex:{fakespace_df['Space'][i]} rdf:type famo:PhysicalSpace."+'\n')

    #record properties
    #contains space(s), if applicable
    if (pd.notna(fakespace_df['ContainsSpace'][i])):
        ttl_list.append(f"ex:wms_spacerec_{fakespace_df['Space'][i]} famo:representsObjectOf ex:wms_spaceincl_{fakespace_df['ContainsSpace'][i]}_{dt_now}."+'\n')
        ttl_list.append(f"ex:wms_spaceincl_{fakespace_df['ContainsSpace'][i]}_{dt_now} a famo:SpaceInclusionRelation."+'\n')
        ttl_list.append(f"ex:wms_spaceincl_{fakespace_df['ContainsSpace'][i]}_{dt_now} famo:subjectIsRepresentedBy ex:wms_spacerec_{fakespace_df['ContainsSpace'][i]}."+'\n')
### HERE ###
    #entity number
    ttl_list.append(f"ex:wms_spacerec_{fakespace_df['Space'][i]} famo:representsSubjectOf ex:wms_spaceid_{fakespace_df['Space'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_spaceincl_{fakespace_df['SpaceNum'][i]}_{dt_now} a famo:IdentificationNumberProperty."+'\n')
    ttl_list.append(f"ex:wms_spaceincl_{fakespace_df['SpaceNum'][i]}_{dt_now} famo:hasValue '{fakespace_df['SpaceNum'][i]}'."+'\n')
    
    #description
    ttl_list.append(f"ex:wms_spacerec_{fakespace_df['Space'][i]} famo:representsSubjectOf ex:wms_desc_{fakespace_df['Space'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_desc_{fakespace_df['Space'][i]}_{dt_now} a famo:NameProperty."+'\n')
    ttl_list.append(f"ex:wms_desc_{fakespace_df['Space'][i]}_{dt_now} famo:hasValue '{fakespace_df['Description'][i]}'."+'\n')

In [5]:
ttl_results = open("01_fakespace_out.ttl", "a")
#for i in range(0,len(fakeasset_df)-1):
#record instantiation and interpretation
ttl_results.writelines(ttl_list)

ttl_results.close()